# Homework on MLE


#####   Estimate a Normal-Half Normal Model.

Consider the Model A below:

\begin{aligned}
 y_i & = \alpha + x_i' \beta + v_i - u_i,\\
 v_i & \sim N(0, \sigma_v^2), \\
 u_i & \sim N^+(0, \sigma_u^2).
\end{aligned}
 
 
The following is Model A's log-likelihood function of the $i$th observation. 

\begin{align}
L_i = - \ln \left(\frac{1}{2}\right) -\frac{1}{2}\ln (\sigma_v^2 + \sigma_u^2) + \ln
\phi\left(\frac{\epsilon_i}{\sqrt{\sigma_v^2 + \sigma_u^2}} \right) +
\ln \Phi\left(\frac{\mu_{*i}}{\sigma_*} \right),
\end{align}

where $\phi(z)$ and $\Phi(z)$ are the PDF and CDF of a standard normal distribution. Also,

\begin{aligned}
 \mu_{*i}  = \frac{-\sigma_u^2 \epsilon_i}{\sigma_v^2 + \sigma_u^2},\qquad
 \sigma_*^2  = \frac{\sigma_v^2  \sigma_u^2}{\sigma_v^2 + \sigma_u^2}. 
\end{aligned}


The attached dataset, `sampledata.csv`, contains data of agricultural production from India. The variables are the follows. They have been converted to appropriate units (taking log, etc.) so no further data processing is necessary.


|          |  |        |
|-------------------------------------|---|---------------------|
| yvar: rice output                   |   | Lland: land         |
| Plland: irrigated land              |   | Llabor: labor       |
| Lbull: bull cost                    |   | Lcost: other costs  |
| yr: production year                 |   | age: age of farmers |
| school: farmers' years of schooling |   | yr_1: same as year  |


The Indian farming model is the follows.

$yvar_i = \alpha + \beta_1 Lland_i + \beta_2 Plland_i + \beta_3 Llabor_i + \beta_4 Lbull_i + \beta_5 Lcost_i + \beta_6 yr_i + v_i - u_i$.
  
You may use the following code to read in the data.

```julia
####################
using DataFrames, CSV
df = DataFrame(CSV.File("sampledata.csv"))
y = df[:, "yvar"]       # the dep var
x = Matrix(df[:, 2:7])  # the indep vars, not including a constant
####################
```

###### Your work: 

- Based on (1), write a program of Model A's log-likelihood function and call it `NHN_mle`. You may use `loglike5` in the lecture note as the reference. 


- Use `NHN_mle` to estimate the Indian farming model. You may conduct the estimation using `Optim` or your own programs. The required result is a table with three columns: the 1st column is the coefficients, the 2nd column is the standard errors, and the 3rd column is the $t$ statistics. They do not have to be in DataFrames.


- The Estimation Guidelines:
  
  - You may use $-0.1$ as the initial values for all parameters. Or you may use the OLS result as the initial values for the $\alpha$ and $\beta$ coefficients (`x2=hcat(ones(size(y,1), 1), x); ols=inv(x2'x2)*(x2'y)`). Or, you may choose any initial values that seem to be reasonable choices. However, you _**cannot**_ use the true answer as the initial values.

  - I strongly suggest that your program uses the `autodiff = :forward` option (which uses automatic differentiation) in the estimation.
  
    - The `autodiff = :forward` option puts stringent requirements on data types which may not be easy to work out. I suggest that you start with your program without the option (which would then default to numerical finite differences which is easier on the data). After you have a working program, you may add the option back and see if it works. Most likely you'll have error messages and you have to work out the issues. If you can't make it work, that's fine. Just try your best.  
  
- Hint: Look at $|g(x)|$ to judge whether it is converged. It should be smaller than the convergence criterion. 

In [1]:
# Grading Notes: Some students' variance parameters are incorrectly estimated. Mistakes in the definition of \sigma_s^*.

using Random, Distributions, Optim, StatsFuns, LinearAlgebra
using DataFrames, CSV
using FLoops


function NHN_mle(y, x, α,  β, log_σᵤ², log_σᵥ² ) # the mle
  σᵤ²  = exp(log_σᵤ²)
  σᵥ²  = exp(log_σᵥ²)
  ϵ  = y .- α .- x * β    

  σ²  = σᵤ² + σᵥ²
  μₛ  = ( - σᵤ² .* ϵ) ./ σ²
  σₛ² = (σᵥ² * σᵤ²) / σ²

#=
  # safer coding   
  llike = Array{Real}(undef, size(y,1))
  for i in eachindex(y) 
    llike[i] = ( - 0.5 * log(σ²) 
             + normlogpdf( (ϵ[i]) / sqrt(σ²))  
             + normlogcdf((μₛ[i]) / sqrt(σₛ²))  
             - normlogcdf(0)  )
  end
  return -sum(llike)
=#    
  
  # faster coding  
  return -sum(- 0.5 * log(σ²) .+ normlogpdf.( ϵ ./ sqrt(σ²)) .+ normlogcdf.(μₛ ./ sqrt(σₛ²)) .- normlogcdf(0))
    
end 


###### import empirical data ########

nofx = 6

df = DataFrame(CSV.File("sampledata.csv"))
y = df[:, "yvar"]       # the dep var
x = Matrix(df[:, 2:nofx+1])  # the indep vars, not including a constant

init = -0.1*ones(nofx+3)

# x2=hcat(ones(size(y,1), 1), x); init = inv(x2'x2)*(x2'y); push!(init, -0.1, -0.1)

##### start estimation #########

func1 = TwiceDifferentiable(vars -> NHN_mle(y, x, vars[1],
                            vars[2:nofx+1], vars[end-1], vars[end]),
                            ones(nofx+3))

res1 = optimize(func1, init, Newton(), 
                Optim.Options(g_tol = 1e-7, 
                              iterations = 2000))
                              

coeff1 = Optim.minimizer(res1)        
hwk13_coeff = deepcopy(coeff1)                # keep _coevec untouched
hwk13_coeff[end-1:end] = exp.(hwk13_coeff[end-1:end])     # convert unit of the last two elements
_Hessian  = Optim.hessian!(func1, coeff1)  # Hessain evaluated at the coeff vector


#=
# The block shows that `isposdef()` may be misleading. Note that 
#   in Optim's convex setting, the Hessian should be positive definite.
#   It is defined by eigen values being positive. However, `isposdef()`
#   also checks that the matrix being symmetric but it often gives 
#   false alarms when there is tiny numerical variations. This is most 
#   likely to happen when `autodiff = :forward` is used. 

aa = eigen(_Hessian)          # See, all the eigenvalues are positive. 
@show aa.values 
@show isposdef(_Hessian)      # But it shows "false" which is incorrect.
@show _Hessian == _Hessian'   # It is because the Hessian is not perfectly symmetric.
@show isapprox(_Hessian, _Hessian'; rtol=1e-12)  # which is "true"
aaaaa
=#

var_cov_matrix = inv(_Hessian)             # don't need the "negative" of Hessian. Why?

stderror  = sqrt.(diag(var_cov_matrix))

stderror[end-1:end] = hwk13_coeff[end-1:end] .*stderror[end-1:end]  # convert the unit using the delta method
t_stats = hwk13_coeff ./ stderror
hwk13_table = hcat(hwk13_coeff, stderror, t_stats)

@show res1

println("The estimation table is")
hwk13_table |> display

res1 =  * Status: success

 * Candidate solution
    Final objective value:     9.336604e+01

 * Found with
    Algorithm:     Newton's Method

 * Convergence measures
    |x - x'|               = 8.18e-08 ≰ 0.0e+00
    |x - x'|/|x'|          = 2.39e-08 ≰ 0.0e+00
    |f(x) - f(x')|         = 7.82e-13 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 8.37e-15 ≰ 0.0e+00
    |g(x)|                 = 5.89e-08 ≤ 1.0e-07

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    33
    f(x) calls:    145
    ∇f(x) calls:   145
    ∇²f(x) calls:  33

The estimation table is


9×3 Matrix{Float64}:
  1.59012     0.349984     4.54342
  0.286194    0.0698685    4.09618
  0.234652    0.176519     1.32933
  1.15546     0.0812037   14.2292
 -0.421433    0.0594057   -7.09415
  0.00661819  0.0130431    0.507411
  0.0340904   0.00803052   4.2451
  0.25651     0.0421415    6.08686
  0.0325692   0.0091596    3.55574

In [3]:
# Using our own program. Not very successful. It is sensitive to initial values.

using LinearAlgebra, ForwardDiff, Random


function newton_max_a(f, init, ϵ, maxit)         # from earlier homework
    iter, Δ, x0, x1 = 1, fill(Inf, length(init)), init, init  # dimensions of f & init should be consistent
    ∇f(x) = ForwardDiff.gradient(f, x)          # define the functions outside the loop
      H(x) = ForwardDiff.hessian(f, x)
    while norm(Δ) > ϵ && iter ≤ maxit            # check the gradient  
        x1 = x0 - inv(H(x0))*∇f(x0)             # inv is from LinearAlgebra  
        Δ, x0, iter = ∇f(x1), x1, iter+1    
    end    
    println("* The solution is $x1, the iteration count is ", iter-1, " and the norm of the gradient is ", norm(Δ), ".")   
    if iter-1 == maxit   # iter number=maxit does not necessarily mean not converged
       println("  The iteration count is exactly equal to maxit, and the estimation may not have converged.") 
    end    
    !isposdef(H(x1)) || println("  The solution appears to be a minimum rather than a maximum.")
    return x1, iter-1, Δ    
end



function NHN_mle2(p) # the mle
    σᵤ²  = exp(p[end-1])
    σᵥ²  = exp(p[end])
    ϵ  = y .- p[1] .- x * p[2:end-2]    

  σ²  = σᵤ² + σᵥ²
  μₛ  = ( - σᵤ² .* ϵ) ./ σ²
  σₛ² = (σᵥ² * σᵤ²) / σ²

  llike = Array{Real}(undef, size(y,1))
  @inbounds for i in eachindex(y)           # @inbounds, @views are not essential
             @views llike[i] = ( - 0.5 * log(σ²) 
             + normlogpdf( (ϵ[i]) / sqrt(σ²))  
             + normlogcdf((μₛ[i]) / sqrt(σₛ²))  
             - normlogcdf(0)  )
  end
  return sum(llike)   # do not give it a minus sign
end 



init = [1.59, 0.29, 0.23, 1.15,-0.42, 0.007, 0.034, log(0.256), log(0.033)]

#=
x2 = hcat(ones(length(y)), x)
init = inv(x2'x2)*(x2'*y)
push!(init, -0.1, -0.1)
=#

# init = -0.1*ones(9)

newton_max_a(NHN_mle2, init, 1e-6, 1000 )

* The solution is [1.5901214050581103, 0.286193653183763, 0.23465202245350636, 1.155461775955525, -0.4214326689944656, 0.006618197844776286, 0.034090371979820386, -1.3605886275793102, -3.4243887069914023], the iteration count is 4 and the norm of the gradient is 9.595020998619309e-12.


([1.5901214050581103, 0.286193653183763, 0.23465202245350636, 1.155461775955525, -0.4214326689944656, 0.006618197844776286, 0.034090371979820386, -1.3605886275793102, -3.4243887069914023], 4, [7.164269177906135e-13, 9.901246489363302e-13, 9.287015600989434e-14, 5.1390003363849246e-12, 4.4222403516869235e-12, 4.4293457790445245e-12, 4.994671343183654e-12, -1.748046152272309e-13, 1.587618925213974e-14])

In [2]:
# Other programs/algorithms

using GalacticOptim, OptimizationOptimJL
using OptimizationNOMAD

init = -0.1*ones(9)

##### local optimizer with autodiff
myf = OptimizationFunction((vars, p) -> NHN_mle(y, x, vars[1], vars[2:7], vars[end-1], vars[end]),
                            Optimization.AutoForwardDiff());
myprob = OptimizationProblem(myf, init); # init_ols);
ha = Optimization.solve(myprob, Newton(), reltol=1e-8, maxiters=2000) 
@show ha.u

##### global optimizer (slow)
myf2 = OptimizationFunction((vars, p) -> NHN_mle(y, x, vars[1], vars[2:7], vars[end-1], vars[end]));
myprob2 = OptimizationProblem(myf2, init); # init_ols);
ha2 = Optimization.solve(myprob2, NOMADOpt(), maxiters=200000); # could use `myprob`, no problem.
@show ha2.u


ha.u = [1.5901214050581036, 0.28619365318376216, 0.23465202245350306, 1.1554617759555266, -0.4214326689944663, 0.006618197844776238, 0.03409037197982055, -1.3605886275793115, -3.4243887069913996]


[ Info: Precompiling GalacticOptim [a75be94c-b780-496d-a8a9-0878b188d577]
┌ Warning: Replacing docs for `SciMLBase.sol :: Union{Tuple, Tuple{D}, Tuple{S}, Tuple{N}, Tuple{T}} where {T, N, S, D}` in module `SciMLBase`
└ @ Base.Docs docs/Docs.jl:240
[ Info: Precompiling OptimizationOptimJL [36348300-93cb-4f02-beb5-3c3902f8871e]
[ Info: Precompiling OptimizationNOMAD [2cab0595-8222-4775-b714-9828e6a9e01b]


BBE OBJ
1 15816.733173
2 15636.804342
3 15109.408565
4 13633.302764
5 10028.838906
6  4089.415334
7   898.728115
9   889.323451
21   793.76892 
27   789.133173
28   723.834946
31   686.40717 
32   586.040111
67   583.84734 
78   583.021894
83   580.645453
92   579.08074 
95   578.711152
97   576.284154
100   570.926351
101   570.08702 
111   568.120115
112   560.866195
124   557.486128
125   550.522528
130   548.252669
131   539.469547
135   538.682361
136   532.051773
137   527.434123
138   513.710388
141   511.734677
142   492.404365
145   491.24031 
146   473.194348
149   466.749336
150   452.828575
155   446.717225
163   436.169367
166   420.054964

BBE OBJ
182   386.661294
226   386.524717
231   373.005074
241   362.101164
242   361.302527
252   358.998387
265   356.211278
277   354.909172
284   352.946262
293   348.817167
322   348.377843
328   345.635322
332   345.098774
352   342.7411  
355   340.056434
356   331.579996
357   317.823609
384   311.470647
386   310.935371
392   3

3946    93.613965
3960    93.611834
3976    93.611686
3979    93.609002
3984    93.604477
4016    93.591192
4019    93.580319
4039    93.569951
4072    93.562015
4116    93.561485
4124    93.559314
4125    93.555128
4126    93.553777
4127    93.55198 
4129    93.551851
4135    93.542189
4137    93.535921
4138    93.52881 
4143    93.527875
4144    93.520137
4146    93.519882
4147    93.514908

BBE OBJ
4154    93.511549
4158    93.505914
4168    93.502497
4172    93.500554
4185    93.492776
4193    93.492341
4203    93.492198
4210    93.482799
4213    93.481254
4219    93.478432
4268    93.475295
4272    93.475176
4277    93.4739  
4281    93.473491
4297    93.466505
4365    93.466156
4370    93.465815
4371    93.465545
4389    93.465285
4392    93.464972
4398    93.461611
4420    93.460671
4437    93.459243
4440    93.454976
4457    93.454464
4477    93.452198
4480    93.448929
4501    93.446259
4502    93.445802
4520    93.442076
4521    93.442045
4530    93.441902
4535    93.441603
4

9-element Vector{Float64}:
  1.58656953788892
  0.2855636255144
  0.23346340072049
  1.15618394209488
 -0.42156032755639
  0.00657680309073
  0.03413858229263
 -1.36103897045444
 -3.42353407483253